# Load data

In [ ]:
import pandas as pd
df_cleaned = pd.read_csv("/content/drive/MyDrive/HateSpeech/TODA(CLEANED)/BERT/TODA_CLEANED_01.csv").iloc[:,1:]
df_cleaned = df_cleaned.dropna()

In [ ]:
df_typo = pd.read_csv("/content/drive/MyDrive/HateSpeech/TODA(CLEANED)/BERT/TODA_TYPO_01.csv").iloc[:,1:]
df_typo = df_typo.dropna()

In [ ]:
# print(df_cleaned.shape, df_typo.shape)
# pd.concat([df_typo.sample(5,random_state=2),df_cleaned.sample(5,random_state=2)]).sort_index()
print(df_typo.shape)
df_typo

(321596, 7)


,text_raw,text_cleaned,typo_corrected,tokens_raw,tokens_stop,tokens_stems,class
0,@BreitbartNews Idiots seem to come out of the ...,idiots seem to come out of the woodwork it h...,idiots seem to come out of the woodwork it h...,"[' ', 'idiots', 'seem', 'to', 'come', 'out', '...","['idiots', 'come', 'woodwork', 'threatens', 'p...","['idiot', 'come', 'woodwork', 'threaten', 'pre...",1
1,russia as a huge black hole where everything d...,russia as a huge black hole where everything d...,russia as a huge black hole where everything d...,"['russia', 'as', 'a', 'huge', 'black', 'hole',...","['russia', 'huge', 'black', 'hole', 'disappear...","['russia', 'huge', 'black', 'hole', 'disappear...",0
2,@user so much stuff happening in florida! firs...,so much stuff happening in florida! first orl...,so much stuff happening in florida! first orl...,"[' ', 'so', 'much', 'stuff', 'happening', 'in'...","['stuff', 'happening', 'florida', 'orlando', '...","['stuff', 'happen', 'florida', 'orlando', 'sho...",0
3,"@user #pennydreadful a long, dark love poem ð...",pennydreadful a long dark love poem i agree...,pennydreadful a long dark love poem i agree...,"[' ', 'pennydreadful', 'a', 'long', 'dark', '...","['pennydreadful', 'long', 'dark', 'love', 'poe...","['pennydread', 'long', 'dark', 'love', 'poem',...",0
4,@user @user @user this is how i wanted to spnd...,this is how i wanted to spnd father's day w...,this is how i wanted to send father's day w...,"[' ', 'this', 'is', 'how', 'i', 'wanted', 't...","['wanted', 'send', 'father', 'day', 'wind', 'b...","['want', 'send', 'father', 'day', 'wind', 'boy...",0
...,...,...,...,...,...,...,...
321711,https://t.co/P4TPfJT2Bb #TreCru https://t.co/g...,trecru,record,"[' ', 'record']",['record'],['record'],0
321712,"whereas East Asians came from East Africa,",whereas east asians came from east africa,whereas east asians came from east africa,"['whereas', 'east', 'asians', 'came', 'from', ...","['east', 'asians', 'came', 'east', 'africa']","['east', 'asian', 'came', 'east', 'africa']",0
321713,@AndrewBLeh @N7Kopper @nitramy Ever play Rambo...,ever play rambo on old nes by any chance?,ever play rambo on old nes by any chance?,"[' ', 'ever', 'play', 'rambo', 'on', 'old', ...","['play', 'rambo', 'old', 'nes', 'chance']","['play', 'rambo', 'old', 'ne', 'chanc']",1
321714,Merger \n\nIf this article is to avoid being m...,merger if this article is to avoid being merge...,merger if this article is to avoid being merge...,"['merger', 'if', 'this', 'article', 'is', 'to'...","['merger', 'article', 'avoid', 'merged', 'hist...","['merger', 'articl', 'avoid', 'merg', 'histori...",0


In [ ]:
X_cleaned = df_typo['text_cleaned']
X_typo = df_typo['typo_corrected']
y = df_typo['class']

In [ ]:
X_cleaned

0          idiots seem to come out of the woodwork  it h...
1         russia as a huge black hole where everything d...
2          so much stuff happening in florida! first orl...
3           pennydreadful a long dark love poem  i agree...
4            this is how i wanted to spnd father's day w...
                                ...                        
321711                                              trecru 
321712            whereas east asians came from east africa
321713            ever play rambo on old nes by any chance?
321714    merger if this article is to avoid being merge...
321715    zoe quinn and other female game developers spe...
Name: text_cleaned, Length: 321596, dtype: object

## KFOLD

In [ ]:
import ast
from sklearn.model_selection import KFold
# K_Fold Cross Validation
df_typo['tokens_raw'] = df_typo['tokens_raw'].apply(lambda x : ast.literal_eval(x))
kf = KFold(n_splits=5,random_state=42)
K_FOLD_LIST = []
k=1
for train_index, test_index in kf.split(df_cleaned):
  K_FOLD_LIST.append([train_index, test_index,k,df_typo['tokens_raw'].iloc[test_index].apply(len).sum()])
  k+=1

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


In [ ]:
print("train index\t\t\t\t\ttest index\t\t\t\t\tk   word nums")
print(K_FOLD_LIST[0][0],K_FOLD_LIST[0][1],"     ",K_FOLD_LIST[0][2],K_FOLD_LIST[0][3])
print(K_FOLD_LIST[1][0],K_FOLD_LIST[1][1],K_FOLD_LIST[1][2],K_FOLD_LIST[1][3])
print(K_FOLD_LIST[2][0],K_FOLD_LIST[2][1],K_FOLD_LIST[2][2],K_FOLD_LIST[2][3])
print(K_FOLD_LIST[3][0],K_FOLD_LIST[3][1],K_FOLD_LIST[3][2],K_FOLD_LIST[3][3])
print(K_FOLD_LIST[4][0],K_FOLD_LIST[4][1],K_FOLD_LIST[4][2],K_FOLD_LIST[4][3])

train index					test index					k   word nums
[ 64320  64321  64322 ... 321593 321594 321595] [    0     1     2 ... 64317 64318 64319]       1 1955231
[     0      1      2 ... 321593 321594 321595] [ 64320  64321  64322 ... 128636 128637 128638] 2 1939953
[     0      1      2 ... 321593 321594 321595] [128639 128640 128641 ... 192955 192956 192957] 3 1951304
[     0      1      2 ... 321593 321594 321595] [192958 192959 192960 ... 257274 257275 257276] 4 1941812
[     0      1      2 ... 257274 257275 257276] [257277 257278 257279 ... 321593 321594 321595] 5 1935521


In [ ]:
max(df_typo['tokens_raw'].apply(len))

235

## GloVe

## Tokenizer

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
max_features=100000
sequence_length = 235

tokenizer_clean = Tokenizer(num_words=max_features, split=' ', oov_token='<unw>', filters=' ')
tokenizer_typo = Tokenizer(num_words=max_features, split=' ', oov_token='<unw>', filters=' ')

tokenizer_clean.fit_on_texts(X_cleaned)
tokenizer_typo.fit_on_texts(X_typo)

# this takes our sentences and replaces each word with an integer
X_clean_token = tokenizer_clean.texts_to_sequences(X_cleaned)
X_typo_token = tokenizer_typo.texts_to_sequences(X_typo)

# we then pad the sequences so they're all the same length (sequence_length)
X_clean_token = pad_sequences(X_clean_token, sequence_length)
X_typo_token = pad_sequences(X_typo_token, sequence_length)

In [ ]:
dataset_list = [X_clean_token,X_typo_token]
dataset_name = ["Clean","Typo"]

dataset = {na:li for na,li in zip(dataset_name,dataset_list)}

## Embedding

In [ ]:
import numpy as np
embeddings_index = {}
f = open("/content/drive/MyDrive/HateSpeech/glove.6B.100d.txt")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [ ]:
word_index_clean = tokenizer_clean.word_index
word_index_typo = tokenizer_typo.word_index
print('[CLEAN]Found %s unique tokens.' % len(word_index_clean))
print('[TYPO ]Found %s unique tokens.' % len(word_index_typo))

[CLEAN]Found 337465 unique tokens.
[TYPO ]Found 264425 unique tokens.


In [ ]:
num_words_clean = min(max_features, len(word_index_clean)) + 1
num_words_typo = min(max_features, len(word_index_typo)) + 1
print("num_words_clean : ",num_words_clean)
print("num_words_typo : ",num_words_typo)

embedding_dim = 100

# first create a matrix of zeros, this is our embedding matrix
embedding_matrix_clean = np.zeros((num_words_clean, embedding_dim))
embedding_matrix_typo = np.zeros((num_words_typo, embedding_dim))

for word, i in word_index_clean.items():
    if i > max_features:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # we found the word - add that words vector to the matrix
        embedding_matrix_clean[i] = embedding_vector
    else:
        # doesn't exist, assign a random vector
        embedding_matrix_clean[i] = np.random.randn(embedding_dim)

for word, i in word_index_typo.items():
    if i > max_features:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix_typo[i] = embedding_vector
    else:
        embedding_matrix_typo[i] = np.random.randn(embedding_dim)

num_words_clean :  100001
num_words_typo :  100001


In [ ]:
embedding_matrix_clean.shape

(100001, 100)

# MODELING

## build LSTM

In [ ]:
from keras.models import Sequential
from keras.initializers import Constant
from keras.layers import *

def build_model(num_words,embedding_matrix):
  model = Sequential()
  model.add(Embedding(num_words,
                      embedding_dim,
                      embeddings_initializer=Constant(embedding_matrix),
                      input_length=sequence_length,
                      trainable=True))
  model.add(SpatialDropout1D(0.2))
  model.add(Bidirectional(CuDNNLSTM(64, return_sequences=True)))
  model.add(Bidirectional(CuDNNLSTM(32)))
  model.add(Dropout(0.25))
  model.add(Dense(units=1, activation='sigmoid'))
  model.compile(loss = 'binary_crossentropy', optimizer='adam',metrics = ['accuracy'])
  return model

model_clean = build_model(num_words_clean,embedding_matrix_clean)
model_typo = build_model(num_words_typo,embedding_matrix_typo)

In [ ]:
model_clean.summary()

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_10 (Embedding)     (None, 235, 100)          10000100  
_________________________________________________________________
spatial_dropout1d_10 (Spatia (None, 235, 100)          0         
_________________________________________________________________
bidirectional_20 (Bidirectio (None, 235, 128)          84992     
_________________________________________________________________
bidirectional_21 (Bidirectio (None, 64)                41472     
_________________________________________________________________
dropout_10 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_10 (Dense)             (None, 1)                 65        
Total params: 10,126,629
Trainable params: 10,126,629
Non-trainable params: 0
_________________________________________

In [ ]:
model_typo.summary()

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_11 (Embedding)     (None, 235, 100)          10000100  
_________________________________________________________________
spatial_dropout1d_11 (Spatia (None, 235, 100)          0         
_________________________________________________________________
bidirectional_22 (Bidirectio (None, 235, 128)          84992     
_________________________________________________________________
bidirectional_23 (Bidirectio (None, 64)                41472     
_________________________________________________________________
dropout_11 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 65        
Total params: 10,126,629
Trainable params: 10,126,629
Non-trainable params: 0
_________________________________________

## Train

In [ ]:
import pandas as pd
import numpy as np
import time
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score

In [ ]:
eval = pd.DataFrame([[np.nan for i in range(11)]])
eval.columns = ['Model','Dataset','K_fold','Train_Score(ACC)','Train_Score(ROC_AUC)','Train_Score(F1)','Test_Score(ACC)','Test_Score(ROC_AUC)','Test_Score(F1)','Inference_Time','words_nums']
eval = eval.iloc[1:]


for data_name in dataset_name:
  # Performance Base DataFrame
  LR_list_base = ['BiLSTM+GloVe(10)']
  LR_list_base.append(data_name)

  # Load Data
  X = dataset[data_name]

  # K_Fold Cross Validation
  for train_index, test_index, k, word_nums in K_FOLD_LIST:
    if data_name=='Clean':
      print("Model : CLEAN")
      LR = build_model(num_words_clean,embedding_matrix_clean)
    elif data_name=='Typo':
      print("Model : Typo")
      LR = build_model(num_words_typo,embedding_matrix_typo)
    else :
      print("**ERROR** Model Not Found")
    # Get Train, Test
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Fit and Inference
    LR.fit(X_train,y_train,epochs=10,batch_size=256)
    print(f"{data_name} - {k} Trained")
    fitted = LR.predict_classes(X_train)
    fitted_proba = LR.predict_proba(X_train)
    start = time.time()
    pred = LR.predict_classes(X_test)
    inference_time = time.time()-start
    pred_proba = LR.predict_proba(X_test)
    print(f"{data_name} - {k} Inferenced : {inference_time}s",end='\t')
    # Evaluate
    train_acc = accuracy_score(y_train.values,fitted)
    train_auc = roc_auc_score(y_train.values,fitted_proba)
    train_f1 = f1_score(y_train.values,fitted)
    test_acc = accuracy_score(y_test.values,pred)
    test_auc = roc_auc_score(y_test.values,pred_proba)
    test_f1 = f1_score(y_test.values,pred)
    print(f"train ACC : {train_acc} test ACC : {test_acc} test F1 : {test_f1}")

    LR_list = LR_list_base.copy()
    LR_list.append(k)
    LR_list.append(train_acc)
    LR_list.append(train_auc)
    LR_list.append(train_f1)
    LR_list.append(test_acc)
    LR_list.append(test_auc)
    LR_list.append(test_f1)
    LR_list.append(inference_time)
    LR_list.append(word_nums)

    eval = eval.append(pd.DataFrame([LR_list],columns=eval.columns))
eval

Model : CLEAN
Epoch 1/10
1005/1005 [==============================] - 134s 128ms/step - loss: 0.3143 - accuracy: 0.8751
Epoch 2/10
1005/1005 [==============================] - 127s 127ms/step - loss: 0.1907 - accuracy: 0.9264
Epoch 3/10
1005/1005 [==============================] - 128s 128ms/step - loss: 0.1683 - accuracy: 0.9348
Epoch 4/10
1005/1005 [==============================] - 129s 128ms/step - loss: 0.1495 - accuracy: 0.9419
Epoch 5/10
1005/1005 [==============================] - 128s 127ms/step - loss: 0.1285 - accuracy: 0.9500
Epoch 6/10
1005/1005 [==============================] - 126s 125ms/step - loss: 0.1143 - accuracy: 0.9555
Epoch 7/10
1005/1005 [==============================] - 126s 125ms/step - loss: 0.0997 - accuracy: 0.9615
Epoch 8/10
1005/1005 [==============================] - 126s 126ms/step - loss: 0.0888 - accuracy: 0.9661
Epoch 9/10
1005/1005 [==============================] - 127s 126ms/step - loss: 0.0767 - accuracy: 0.9712
Epoch 10/10
1005/1005 [=========

/usr/local/lib/python3.7/dist-packages/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '
/usr/local/lib/python3.7/dist-packages/keras/engine/sequential.py:425: UserWarning: `model.predict_proba()` is deprecated and will be removed after 2021-01-01. Please use `model.predict()` instead.
  warnings.warn('`model.predict_proba()` is deprecated and '


Clean - 1 Inferenced : 50.11148977279663s	train ACC : 0.9839161056608468 test ACC : 0.9235385572139303 test F1 : 0.81185921958684
Model : CLEAN
Epoch 1/10
1005/1005 [==============================] - 129s 125ms/step - loss: 0.3128 - accuracy: 0.8766
Epoch 2/10
1005/1005 [==============================] - 124s 124ms/step - loss: 0.1947 - accuracy: 0.9262
Epoch 3/10
1005/1005 [==============================] - 124s 124ms/step - loss: 0.1696 - accuracy: 0.9348
Epoch 4/10
1005/1005 [==============================] - 125s 124ms/step - loss: 0.1480 - accuracy: 0.9432
Epoch 5/10
1005/1005 [==============================] - 125s 125ms/step - loss: 0.1305 - accuracy: 0.9495
Epoch 6/10
1005/1005 [==============================] - 125s 124ms/step - loss: 0.1147 - accuracy: 0.9562
Epoch 7/10
1005/1005 [==============================] - 126s 126ms/step - loss: 0.1017 - accuracy: 0.9610
Epoch 8/10
1005/1005 [==============================] - 126s 125ms/step - loss: 0.0891 - accuracy: 0.9657
Epoch 9/

/usr/local/lib/python3.7/dist-packages/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '
/usr/local/lib/python3.7/dist-packages/keras/engine/sequential.py:425: UserWarning: `model.predict_proba()` is deprecated and will be removed after 2021-01-01. Please use `model.predict()` instead.
  warnings.warn('`model.predict_proba()` is deprecated and '


Clean - 2 Inferenced : 48.75499415397644s	train ACC : 0.9825752010478978 test ACC : 0.9269578196178423 test F1 : 0.8218024578971326
Model : CLEAN
Epoch 1/10
1005/1005 [==============================] - 128s 124ms/step - loss: 0.3176 - accuracy: 0.8750
Epoch 2/10
1005/1005 [==============================] - 124s 124ms/step - loss: 0.1906 - accuracy: 0.9279
Epoch 3/10
1005/1005 [==============================] - 124s 124ms/step - loss: 0.1692 - accuracy: 0.9358
Epoch 4/10
1005/1005 [==============================] - 126s 125ms/step - loss: 0.1479 - accuracy: 0.9426
Epoch 5/10
1005/1005 [==============================] - 126s 126ms/step - loss: 0.1290 - accuracy: 0.9499
Epoch 6/10
1005/1005 [==============================] - 126s 125ms/step - loss: 0.1130 - accuracy: 0.9570
Epoch 7/10
1005/1005 [==============================] - 126s 125ms/step - loss: 0.0988 - accuracy: 0.9621
Epoch 8/10
1005/1005 [==============================] - 126s 126ms/step - loss: 0.0873 - accuracy: 0.9674
Epoch 

...

/usr/local/lib/python3.7/dist-packages/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '
/usr/local/lib/python3.7/dist-packages/keras/engine/sequential.py:425: UserWarning: `model.predict_proba()` is deprecated and will be removed after 2021-01-01. Please use `model.predict()` instead.
  warnings.warn('`model.predict_proba()` is deprecated and '


Typo - 4 Inferenced : 47.15133762359619s	train ACC : 0.9810904200530945 test ACC : 0.9194639220137129 test F1 : 0.8001851566116339
Model : Typo
Epoch 1/10
1005/1005 [==============================] - 130s 126ms/step - loss: 0.3197 - accuracy: 0.8737
Epoch 2/10
1005/1005 [==============================] - 125s 124ms/step - loss: 0.2014 - accuracy: 0.9226
Epoch 3/10
1005/1005 [==============================] - 124s 123ms/step - loss: 0.1771 - accuracy: 0.9306
Epoch 4/10
1005/1005 [==============================] - 127s 126ms/step - loss: 0.1563 - accuracy: 0.9386
Epoch 5/10
1005/1005 [==============================] - 126s 125ms/step - loss: 0.1383 - accuracy: 0.9453
Epoch 6/10
1005/1005 [==============================] - 126s 125ms/step - loss: 0.1208 - accuracy: 0.9526
Epoch 7/10
1005/1005 [==============================] - 126s 125ms/step - loss: 0.1074 - accuracy: 0.9581
Epoch 8/10
1005/1005 [==============================] - 126s 125ms/step - loss: 0.0938 - accuracy: 0.9638
Epoch 9/

/usr/local/lib/python3.7/dist-packages/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '
/usr/local/lib/python3.7/dist-packages/keras/engine/sequential.py:425: UserWarning: `model.predict_proba()` is deprecated and will be removed after 2021-01-01. Please use `model.predict()` instead.
  warnings.warn('`model.predict_proba()` is deprecated and '


Typo - 5 Inferenced : 47.887691497802734s	train ACC : 0.9825829747703837 test ACC : 0.9156703306954399 test F1 : 0.7996601905887568


,Model,Dataset,K_fold,Train_Score(ACC),Train_Score(ROC_AUC),Train_Score(F1),Test_Score(ACC),Test_Score(ROC_AUC),Test_Score(F1),Inference_Time,words_nums
0,BiLSTM+GloVe(10),Clean,1.0,0.983916,0.998228,0.961423,0.923539,0.949479,0.811859,50.111490,1955231.0
0,BiLSTM+GloVe(10),Clean,2.0,0.982575,0.997885,0.958213,0.926958,0.950719,0.821802,48.754994,1939953.0
0,BiLSTM+GloVe(10),Clean,3.0,0.984585,0.998214,0.963099,0.923786,0.949220,0.814487,48.175798,1951304.0
0,BiLSTM+GloVe(10),Clean,4.0,0.983403,0.998130,0.960163,0.922760,0.948057,0.811146,48.216493,1941812.0
0,BiLSTM+GloVe(10),Clean,5.0,0.984503,0.998302,0.962836,0.923320,0.947658,0.811396,47.749396,1935521.0
0,BiLSTM+GloVe(10),Typo,1.0,0.981677,0.997707,0.955973,0.921455,0.944601,0.806585,51.332010,1955231.0
0,BiLSTM+GloVe(10),Typo,2.0,0.982136,0.997643,0.957404,0.919355,0.945228,0.806231,48.417917,1939953.0
0,BiLSTM+GloVe(10),Typo,3.0,0.982871,0.997831,0.959051,0.921283,0.945408,0.808850,48.158283,1951304.0
0,BiLSTM+GloVe(10),Typo,4.0,0.981090,0.997739,0.954449,0.919464,0.941540,0.800185,47.151338,1941812.0
0,BiLSTM+GloVe(10),Typo,5.0,0.982583,0.997869,0.958819,0.915670,0.942256,0.799660,47.887691,1935521.0


In [ ]:
eval.to_csv("/content/drive/MyDrive/HateSpeech/PERFORMANCE/bilstm_performance1.csv")

In [ ]:
eval

,Model,Dataset,K_fold,Train_Score(ACC),Train_Score(ROC_AUC),Train_Score(F1),Test_Score(ACC),Test_Score(ROC_AUC),Test_Score(F1),Inference_Time,words_nums
0,BiLSTM+GloVe(10),Clean,1.0,0.983916,0.998228,0.961423,0.923539,0.949479,0.811859,50.111490,1955231.0
0,BiLSTM+GloVe(10),Clean,2.0,0.982575,0.997885,0.958213,0.926958,0.950719,0.821802,48.754994,1939953.0
0,BiLSTM+GloVe(10),Clean,3.0,0.984585,0.998214,0.963099,0.923786,0.949220,0.814487,48.175798,1951304.0
0,BiLSTM+GloVe(10),Clean,4.0,0.983403,0.998130,0.960163,0.922760,0.948057,0.811146,48.216493,1941812.0
0,BiLSTM+GloVe(10),Clean,5.0,0.984503,0.998302,0.962836,0.923320,0.947658,0.811396,47.749396,1935521.0
0,BiLSTM+GloVe(10),Typo,1.0,0.981677,0.997707,0.955973,0.921455,0.944601,0.806585,51.332010,1955231.0
0,BiLSTM+GloVe(10),Typo,2.0,0.982136,0.997643,0.957404,0.919355,0.945228,0.806231,48.417917,1939953.0
0,BiLSTM+GloVe(10),Typo,3.0,0.982871,0.997831,0.959051,0.921283,0.945408,0.808850,48.158283,1951304.0
0,BiLSTM+GloVe(10),Typo,4.0,0.981090,0.997739,0.954449,0.919464,0.941540,0.800185,47.151338,1941812.0
0,BiLSTM+GloVe(10),Typo,5.0,0.982583,0.997869,0.958819,0.915670,0.942256,0.799660,47.887691,1935521.0


In [ ]:
df = pd.read_csv("/content/trial_en.tsv",sep='\t')

In [ ]:
import re
def preprocess(text):
  # URL 제거
  text = re.sub('http\S+', '', text)
  # 멘션 제거
  text = re.sub('@\S+', '', text)
  # 해쉬 및 특수문자 제거
  delete_e = re.compile("[^a-zA-Z0-9\!\?\$\%\' ]")
  text = delete_e.sub("",text)
  text = text.lower()
  return text

In [ ]:
df['cleaned'] = df['text'].apply(preprocess)

In [ ]:
df

,id,text,HS,TR,AG,prediction,cleaned
0,1,RT @vaintshit: shut the fuck up and come suck...,1,1,1,1,rt shut the fuck up and come suck my dick
1,2,@ArianasBotch Ok if you fucking said leave blo...,1,1,1,0,ok if you fucking said leave block me but dm ...
2,3,@CyV_SW Wow mo cock got hard. Want to pull you...,1,1,1,1,wow mo cock got hard want to pull your pantie...
3,4,Ill kill the bitch (chloe) when your not home ...,1,1,1,0,ill kill the bitch chloe when your not home i...
4,5,...............................'I get to rape ...,1,0,1,0,'i get to rape beautiful women and that's why ...
...,...,...,...,...,...,...,...
95,96,"Russia has said that within the coming months,...",0,0,0,0,russia has said that within the coming months ...
96,97,Orthodox Church attacked in hate crime against...,0,0,0,0,orthodox church attacked in hate crime against...
97,98,UK MPs probe unpublished UN â€˜sex-for-foodâ€™...,0,0,0,0,uk mps probe unpublished un sexforfood finding...
98,99,"\""Large areas have been evacuated and people w...",0,0,0,0,large areas have been evacuated and people wer...


In [ ]:
df['HS'].value_counts()

1    50
0    50
Name: HS, dtype: int64

In [ ]:
df['prediction'] = LR.predict_classes(pad_sequences(tokenizer_clean.texts_to_sequences(df['cleaned']), sequence_length))

/usr/local/lib/python3.7/dist-packages/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [ ]:
df['prediction'].value_counts()

1    60
0    40
Name: prediction, dtype: int64

In [ ]:
f1_score(df['HS'],df['prediction'])

0.5818181818181818

In [ ]:
accuracy_score(df['HS'],df['prediction'])

0.54

In [ ]:
a = df_typo[df_typo['class']==1]['class'][15000:20000]
b = LR.predict_classes(pad_sequences(tokenizer_clean.texts_to_sequences(df_typo[df_typo['class']==1]['text_cleaned'][15000:20000]), sequence_length))

/usr/local/lib/python3.7/dist-packages/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [ ]:
accuracy_score(a,b)

0.779

In [ ]:
f1_score(a,b)

0.8757729061270377

In [ ]:
LR.fit(pad_sequences(tokenizer_clean.texts_to_sequences(pd.concat([df_typo['text_cleaned'][:30000],df_typo[df_typo['class']==1]['text_cleaned'][:30000]])), sequence_length),pd.concat([df_typo['class'][:30000],df_typo[df_typo['class']==1]['class'][:30000]]),batch_size=256)

235/235 [==============================] - 30s 128ms/step - loss: 0.2270 - accuracy: 0.9145


# Performance

In [ ]:
import glob
per_list = glob.glob("/content/drive/MyDrive/HateSpeech/PERFORMANCE/*.csv")[:-1]
per_list

['/content/drive/MyDrive/HateSpeech/PERFORMANCE/logistic_performance.csv',
 '/content/drive/MyDrive/HateSpeech/PERFORMANCE/randomforest_performance.csv',
 '/content/drive/MyDrive/HateSpeech/PERFORMANCE/lgbm_performance.csv',
 '/content/drive/MyDrive/HateSpeech/PERFORMANCE/bilstm_performance.csv']

In [ ]:
df = pd.read_csv(per_list[0]).iloc[:,1:]
for d in per_list[1:]:
  df = pd.concat([df,pd.read_csv(d).iloc[:,1:]])
df

,Model,Dataset,K_fold,Train_Score(ACC),Train_Score(ROC_AUC),Train_Score(F1),Test_Score(ACC),Test_Score(ROC_AUC),Test_Score(F1),Inference_Time,words_nums
0,LogisticRegression,Clean_Raw,1.0,0.928011,0.962799,0.810358,0.924580,0.952188,0.799338,0.004323,1955231.0
1,LogisticRegression,Clean_Raw,2.0,0.927821,0.962503,0.809421,0.924735,0.953691,0.801183,0.003997,1939953.0
2,LogisticRegression,Clean_Raw,3.0,0.928058,0.962937,0.810174,0.923460,0.952043,0.798047,0.004046,1951304.0
3,LogisticRegression,Clean_Raw,4.0,0.927852,0.962681,0.809932,0.923615,0.952920,0.797960,0.004014,1941812.0
4,LogisticRegression,Clean_Raw,5.0,0.928365,0.963037,0.811184,0.922947,0.951012,0.796167,0.003793,1935521.0
...,...,...,...,...,...,...,...,...,...,...,...
5,BiLSTM+GloVe(10),Typo,1.0,0.981677,0.997707,0.955973,0.921455,0.944601,0.806585,51.332010,1955231.0
6,BiLSTM+GloVe(10),Typo,2.0,0.982136,0.997643,0.957404,0.919355,0.945228,0.806231,48.417917,1939953.0
7,BiLSTM+GloVe(10),Typo,3.0,0.982871,0.997831,0.959051,0.921283,0.945408,0.808850,48.158283,1951304.0
8,BiLSTM+GloVe(10),Typo,4.0,0.981090,0.997739,0.954449,0.919464,0.941540,0.800185,47.151338,1941812.0


In [ ]:
df.groupby(['Model','Dataset']).mean().sort_values("Test_Score(F1)",ascending=False)

K_fold  ...  words_nums
Model                  Dataset                 ...            
RandomForestClassifier Clean_Stemming     3.0  ...   1944764.2
BiLSTM+GloVe(10)       Clean              3.0  ...   1944764.2
RandomForestClassifier Clean_Stopword     3.0  ...   1944764.2
                       Clean_Raw          3.0  ...   1944764.2
LGBMClassifier         Clean_Stemming     3.0  ...   1944764.2
BiLSTM+GloVe(10)       Typo               3.0  ...   1944764.2
RandomForestClassifier Typo_Stemming      3.0  ...   1944764.2
LogisticRegression     Clean_Stemming     3.0  ...   1944764.2
RandomForestClassifier Typo_Raw           3.0  ...   1944764.2
                       Typo_Stopword      3.0  ...   1944764.2
LogisticRegression     Clean_Stopword     3.0  ...   1944764.2
                       Clean_Raw          3.0  ...   1944764.2
LGBMClassifier         Clean_Raw          3.0  ...   1944764.2
                       Clean_Stopword     3.0  ...   1944764.2
                       Typo_Stemming      3.0  ...   1944764.2
LogisticRegression     Typo_Stemming      3.0  ...   1944764.2
                       Typo_Stopword      3.0  ...   1944764.2
                       Typo_Raw           3.0  ...   1944764.2
LGBMClassifier         Typo_Raw           3.0  ...   1944764.2
                       Typo_Stopword      3.0  ...   1944764.2

[20 rows x 9 columns]

In [ ]:
eval = pd.DataFrame([[np.nan for i in range(11)]])
eval.columns = ['Model','Dataset','K_fold','Train_Score(ACC)','Train_Score(ROC_AUC)','Train_Score(F1)','Test_Score(ACC)','Test_Score(ROC_AUC)','Test_Score(F1)','Inference_Time','words_nums']
eval = eval.iloc[1:]


for data_name in dataset_name:
  for tokenizer, model_seq, model_name, pretrained, plat in Model_list:
    # Performance Base DataFrame
    LR_list_base = [model_name] # ALBERT
    LR_list_base.append(data_name) # CLEAN

    # Load Data
    X = dataset[data_name]

    # K_Fold Cross Validation
    for train_index, test_index, k, word_nums in K_FOLD_LIST:
      tokenizer = tokenizer.from_pretrained(pretrained)
      LR = model.from_pretrained(pretrained)
      optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
      model.compile(optimizer=optimizer, loss=model.compute_loss) # can also use any keras loss fn


      # Get Train, Test
      X_train, X_test = X.iloc[train_index].to_list(), X.iloc[test_index].to_list()
      y_train, y_test = y.iloc[train_index].to_list(), y.iloc[test_index].to_list()

      train_encodings = tokenizer(X_train, truncation=True, padding=True)
      test_encodings = tokenizer(X_test, truncation=True, padding=True)

      train_dataset = tf.data.Dataset.from_tensor_slices((
          dict(train_encodings),
          y_train
      ))
      test_dataset = tf.data.Dataset.from_tensor_slices((
          dict(test_encodings),
          y_test
      ))

      # Fit and Inference
      model.fit(train_dataset.shuffle(1000).batch(16), epochs=1, batch_size=16)

      print(f"{data_name} - {k} Trained")
      fitted = LR.predict(train_dataset.shuffle(1000).batch(16))
      fitted = tf.nn.softmax(fitted.logits, axis=1)
      fitted_proba = fitted[:,1]
      fitted = [np.argmax(res) for res in fitted]
      start = time.time()
      pred = LR.predict(test_dataset.shuffle(1000).batch(16))
      pred = tf.nn.softmax(pred.logits, axis=1)
      inference_time = time.time()-start
      pred_proba = pred[:,1]
      pred = [np.argmax(res) for res in pred_proba]
      print(f"{data_name} - {k} Inferenced : {inference_time}s",end='\t')
      # Evaluate
      train_acc = accuracy_score(y_train,fitted)
      train_auc = roc_auc_score(y_train,fitted_proba)
      train_f1 = f1_score(y_trains,fitted)
      test_acc = accuracy_score(y_test,pred)
      test_auc = roc_auc_score(y_test,pred_proba)
      test_f1 = f1_score(y_test,pred)
      print(f"train ACC : {train_acc} test ACC : {test_acc} test F1 : {test_f1}")

      LR_list = LR_list_base.copy()
      LR_list.append(k)
      LR_list.append(train_acc)
      LR_list.append(train_auc)
      LR_list.append(train_f1)
      LR_list.append(test_acc)
      LR_list.append(test_auc)
      LR_list.append(test_f1)
      LR_list.append(inference_time)
      LR_list.append(word_nums)

      eval = eval.append(pd.DataFrame([LR_list],columns=eval.columns))
eval